In [1]:
# Dashboard libraries
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output

# Data and plots libraries
import sqlite3
import pandas as pd
import plotly.express as px

In [2]:
# Data gathering
df = pd.read_csv("data2.TXT", sep=";")
df = df.drop(["Temp1-18B20 (°C)", "Conc. gas-MQ135 (ppm)"], axis=1)
df["Data/Hora"] = pd.to_datetime(df["Data/Hora"])
df.index = df["Data/Hora"]

# The data I'm using seems to be consistent only between 05-13-2021 and 05-31-2021
# Filter days before
def FiltrarDiaB(df, dia, mes, ano):
    return df.loc[df.index <= pd.Timestamp("{}-{}-{}".format(ano, mes, dia))]

# Filter days after
def FiltrarDiaA(df, dia, mes, ano):
    return df.loc[df.index >= pd.Timestamp("{}-{}-{}".format(ano, mes, dia))]

df = FiltrarDiaA(df, 13, 5, 2021)
df = FiltrarDiaB(df, 31, 5, 2021)

In [3]:
# App inicialization
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

In [4]:
# App Components
Header1 = html.H1('GeniIoT Dashboard', 
                  style={'textAlign': 'center'}) # App title

# Tabs of the tab component
TabTemperature = dcc.Tab(label='Temperature (ºC)', value='TabTemperature')
TabLuminosity = dcc.Tab(label='Luminosity (Lux)', value='TabLuminosity')
TabCO2 = dcc.Tab(label='CO2 (ppm)', value='TabCO2')
TabHumity = dcc.Tab(label='Humidity (%)', value='TabHumity')
TabAC = dcc.Tab(label='AC Use (A)', value='TabAC')

# Tabs component
Tabs = dcc.Tabs(id="tabs-example-graph", 
                value='TabTemperature', 
                children=[TabTemperature, TabLuminosity, TabCO2, TabHumity, TabAC])

# Graph placeholder for tabs
placeholder = html.Div(id='tabs-content-graph')

# Slider Widget
dateRange = list(range(min(df.index.day), max(df.index.day), 3))
dates = df.index.strftime('%m/%d/%Y').unique()
marks = dict(zip(dateRange, dates))

yearSlider = dcc.RangeSlider(min(df.index.day), max(df.index.day), 1,
                             marks=marks,
                             value=[min(df.index.day), max(df.index.day)], 
                             id='year-range-slider')


msTemp = dcc.Dropdown(id='temp-select',
                      options=[{'label': 'Door', 'value': 'Temp3-18B20 (°C)'},
                              {'label': 'Window', 'value': 'Temp4-18B20 (°C)'},
                              {'label': 'Center', 'value': 'Temp2-18B20 (°C)'}],
                      value=["Temp3-18B20 (°C)"],
                      multi=True,
                      style={'width': '50%','padding': 10})


msHum = dcc.Dropdown(id='hum-select',
                     options=[{'label': 'Door', 'value': 'Umid1-dht22 (%)'},
                              {'label': 'Window', 'value': 'Umid2-dht22 (%)'},
                              {'label': 'Center', 'value': 'Umid3-dht22 (%)'}],
                     value=["Umid1-dht22 (%)"],
                     multi=True,
                     style={'width': '50%','padding': 10})

multiGraphs = html.Div(id='multi-graphs')

selectors = html.Div(id='selectors',
                    children=[msTemp, msHum],
                    style={'display': 'flex'})

rcb_graph = html.Div(id="rcb-graph",)

msRcb = dcc.Dropdown(id='rcb-select',
                     options=["ThermA", "IlumA"],
                     value=["ThermA"],
                     multi=True)



# Define app layout
app.layout = html.Div([Header1, Tabs, placeholder, 
                       yearSlider, selectors, multiGraphs, 
                       rcb_graph, msRcb])

In [5]:
# Tabs callback
@app.callback(Output('tabs-content-graph', 'children'),
              [Input('tabs-example-graph', 'value'),
              Input('year-range-slider', 'value')])
def render_content(tab, slider_value):
    # Dictionary {keys -> tab name      values -> y axis values}
    yAxis = {'TabTemperature': "Temp2-18B20 (°C)",
            'TabLuminosity': "Luminosidade (lux)",
            'TabCO2': "Conc. CO2 (ppm)",
            'TabHumity': "Umid2-dht22 (%)",
            'TabAC': ["Corrente1 (A)", "Corrente2 (A)"]}
    
    # Data Frame filtered by date slider
    df2 = FiltrarDiaA(df, slider_value[0], 5, 2021)
    df2 = FiltrarDiaB(df2, slider_value[1], 5, 2021)
    
    fig = px.line(df2, y=yAxis[tab], x="Data/Hora") # Graph made by pyplot
    
    TabContent = html.Div([html.H3('Temperature', 
                                       style={'textAlign': 'center'}), # A title
                               dcc.Graph(figure=fig)]) # The graph
    
    return TabContent



@app.callback(Output('multi-graphs', 'children'),
             [Input('temp-select', 'value'),
              Input('hum-select', 'value'),
              Input('year-range-slider', 'value')])
def render_temp_graph(selTemp, selHum, slider_value):
    
     # Data Frame filtered by date slider
    df2 = FiltrarDiaA(df, slider_value[0], 5, 2021)
    df2 = FiltrarDiaB(df2, slider_value[1], 5, 2021)
    
    figTemp = px.line(df2, y=selTemp, x="Data/Hora")
    figHum = px.line(df2, y=selHum, x="Data/Hora")
    
    content = [dcc.Graph(figure=figTemp, style={'display': 'inline-block'}), 
               dcc.Graph(figure=figHum, style={'display': 'inline-block'})]
    
    return content


@app.callback(Output('rcb-graph', 'children'),
             Input('rcb-select', 'value'))
def update_rcb_graph(value):
    # Different data
    con_data = sqlite3.connect("Saida.db")

    id_rad_ = 12
    query = f'''
    SELECT data, IlumA, ThermA, COALESCE (id_rad1, id_rad2) as id_radiuino FROM (
    (SELECT distinct data FROM beneficio_anualizado)
    
    LEFT JOIN 
    (SELECT data, id_radiuino as id_rad1, valor as IlumA 
    FROM beneficio_anualizado 
    WHERE tipo = 1 and id_radiuino = {id_rad_}) 
    USING (data)
    
    LEFT JOIN 
    (SELECT data, id_radiuino as id_rad2, valor as ThermA 
    FROM beneficio_anualizado 
    WHERE tipo = 2 and id_radiuino = {id_rad_}) 
    USING (data));
    '''
    
    rcb = pd.read_sql_query(query, con=con_data)
    
    rcb_fig = px.scatter(rcb, x="data", y=value)
    
    return dcc.Graph(figure=rcb_fig)

In [6]:
app.run_server()

Dash app running on http://127.0.0.1:8050/


In [8]:
li = ["a", "B"]
dic = {"a":1, "B":2}

a = [dic[i] for i in li]
print(a)

[1, 2]
